In [1]:
import pandas as pd
from pandas import Series,DataFrame,ExcelWriter
import requests
import re
import io
import datetime as dt

import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')

#useful docs for pandas
#http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html

In [2]:
def scrape_irad_data(start_dt, end_dt, report_number):
#scarpes irad data
#input data formats:
    #start_dt, end_dt like 7/1/2016 
    import pandas as pd
    import requests
    import re

    if report_number == 131:
        report_name = '###'
        report_session_url = 'https://irad.ilshealth.com/Report.aspx?From=%s&To=%s&RC=/%s.aspx&AgencyID=45' % (start_dt, end_dt, report_name) 
    elif report_number == 182:
        report_name = '###'
        report_session_url = 'https://irad.ilshealth.com/Report.aspx?FromDate=%s&ToDate=%s&RC=/%s.aspx&AgencyID=45&MemberStatus=ALL' % (start_dt, end_dt, report_name)
    elif report_number == 215:
        report_name = '###'
        report_session_url = 'https://irad.ilshealth.com/Report.aspx?From=%s&To=%s&RC=/%s.aspx&AgencyID=45&ProgramType=5' % (start_dt, end_dt, report_name)
    elif report_number == 274:
        report_name = '###'
        report_session_url = 'https://irad.ilshealth.com/Report.aspx?From=%s&To=%s&RC=/%s.aspx&AgencyID=45' % (start_dt, end_dt, report_name)
    elif report_number == '011':
        report_name = '###'
        report_session_url = 'https://irad.ilshealth.com/Report.aspx?RC=/%s.aspx&AgencyID=45' % (report_name)
    else:
        raise ValueError('Parameter is not valid')
        
        
    with requests.Session() as s:
        login_url = 'https://irad.ilshealth.com/Login.aspx'
        login_html = s.get(login_url, verify=False)

        viewstate_regex = '(?:<input type="hidden" name="__VIEWSTATE" id="__VIEWSTATE" value=")(.*?)(?:" \/>)'
        viewstate = re.findall(viewstate_regex, login_html.text)[0]
        eventvalidation_regex = '(?:<input type="hidden" name="__EVENTVALIDATION" id="__EVENTVALIDATION" value=")(.*?)(?:" \/>)'
        eventvalidation = re.findall(eventvalidation_regex, login_html.text)[0]

        payload = {
        'txtUser': '####',
        'txtPassword': '####',
        'btnLogin': 'Login',
        '__VIEWSTATE': viewstate,
        '__EVENTVALIDATION':eventvalidation
        }
        p = s.post(login_url, data=payload,  verify=False)

        report_session_html = s.get(report_session_url, verify=False)
        ReportSession_regex = '(?:ReportSession=)(.*?)(?:&)'
        ReportSession = re.findall(ReportSession_regex, report_session_html.text)[0]
        ControlID_regex = '(?:ControlID=)(.*?)(?:&)'
        ControlID = re.findall(ControlID_regex, report_session_html.text)[0]
        
        report_url = '###' % (ReportSession, ControlID, report_name)
        r = s.get(report_url, verify=False)
    r = r.content.splitlines()[3:]
    r = '\n'.join(r).decode('utf-8')
    return r

In [ ]:
#pulls irad report 011 - MLTC_Open_Auths
d = scrape_irad_data('', '', '011')  
cr = io.StringIO(d)
df_irad = pd.read_csv(cr)

time_fields = ['MemberTerminationDate', 'AuthTermDate']
for field in time_fields:
    df_irad[field] = pd.to_datetime(df_irad[field], errors='coerce')
    
irad_respites = df_irad[df_irad['cptCode'].isin(['T1005','S5150'])] #respites only
irad_actives = irad_respites[(irad_respites['MemberTerminationDate'].isnull()) | (irad_respites['MemberTerminationDate'] >= dt.datetime.strptime("10/01/2016","%m/%d/%Y"))] #active members
irad_actives = irad_actives[(irad_actives['AuthTermDate'].isnull()) | (irad_actives['AuthTermDate'] >= dt.datetime.strptime("10/01/2016","%m/%d/%Y"))] #active auths
irad_actives = irad_actives[irad_respites['Voided'] == 'No']
irad_actives.loc[irad_actives.cptCode == 'S5150', 'Units'] = irad_actives.Units[irad_actives.cptCode == 'S5150']/4 #converts 15min to 1hr unit
irad_trimmed = irad_actives[['MemberName', 'PlanID', 'Medicaid', 'cptCode', 'Units']]
irad_trimmed =irad_trimmed.rename(columns = {'Units':'hours_authed'})

In [ ]:
#access DB from PPL saved as CSV 
cr = '####'
df_PPL_claims = pd.read_csv(cr)

df_PPL_claims = df_PPL_claims[df_PPL_claims['Service Code'] == 'S5150']
df_PPL_claims = df_PPL_claims.groupby(['Member ID']).Units.sum()
df_PPL_claims = pd.DataFrame({'Member ID':df_PPL_claims.index, 'hours claimed':df_PPL_claims.values})

#claims as CSV
cr = '####'
df_T1005_claims = pd.read_csv(cr)
df_T1005_claims.columns = ['Member ID', 'hours claimed']

df_claims = pd.concat([df_T1005_claims, df_PPL_claims])

In [ ]:
df_final = pd.merge(irad_trimmed, df_claims, left_on='PlanID', right_on='Member ID', how='left')
del df_final['Member ID']

In [6]:
writer = ExcelWriter('output.xlsx')
df_final.to_excel(writer, index=False)
writer.save()